In [ ]:
#1.
    #a. The weight will depend on the 71st observation, which got its weight from the 70th observation. 
        #Adaboost is sequential so it goes 1-by-1
    #b. It gets assigned a larger weight so it will get more attention 
        #in the sequence. If it is classified correctly, it gets a lower
        #weight.
#2.
    #It's an ensable learner becasue it combines weak learners into a
    #more powerful learner to decreases training errors. This algorithm
    #specializes the learners at each step. Things that may need to be adjusted
    #are the estimators and learning rate. 
#3. On Paper!
#4. You could increase the number of estimators or increase the learning rate
#5. a and c
    #a.It usually has lower bias than a single decision tree
    #c.It assigns higher weights to observations that have been misclassified.
#6. f. b and c
    #b. In gradient boosting trees, the decision trees are dependent of each other.
    #c. It is a method for improving the performance by aggregating the results of several decision
        #trees.
#7.         
    #I think the main difference between the two boosting frameworks is that 
    #Ada Boost uses estimators and weights to make the best predictions where Gradient
    #boosting uses the residuls and tries to reduce the error as small as possible.
    #Both use a sequential process. 

In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,  GradientBoostingClassifier 
from sklearn.tree import DecisionTreeClassifier

s3= boto3.resource('s3')
bucket_name= 'morgangant-bata-445-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'framingham.csv'

bucket_object= bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datefile
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [3]:
heart= heart.dropna()

In [7]:
rf_recall= list()
ada_recall= list()
gb_recall= list()

for i in range (0,100):
    #Define the input and target variable
    x= heart[['age', 'totChol','BMI', 'heartRate', 'sysBP', 'glucose']]
    y= heart['TenYearCHD']

    #Splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)
    
    #Buidling the model
    rf_md= RandomForestClassifier(n_estimators= 500, max_depth= 3).fit(x_train, y_train)
    #Predicting on the model
    rf_pred= rf_md.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    rf_labels= np.where(rf_pred < .1, 0, 1)
    rf_recall.append(recall_score(y_test, rf_labels))
    
    #Building the model
    ada_md= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth= 3), n_estimators= 500, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    ada_pred= ada_md.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    ada_labels= np.where(ada_pred < .1, 0, 1)
    ada_recall.append(recall_score(y_test, ada_labels))
    
    #Building the model
    gb_md= GradientBoostingClassifier(max_depth= 3, n_estimators= 500, learning_rate=.01).fit(x_train, y_train)
    #Predicting on the model
    gb_pred= gb_md.predict_proba(x_test)[:,1]
    #changing liklihoods to labels 
    gb_labels= np.where(gb_pred < .1, 0, 1)
    gb_recall.append(recall_score(y_test, gb_labels))

In [8]:
print('Average Recall for Random Forest model:', np.mean(rf_recall))
print('Average Recall for Ada Boost model:',np.mean(ada_recall))
print('Average Recall for Gradient Boost:',np.mean(gb_recall))

Average Recall for Random Forest model: 0.8545535714285716
Average Recall for Ada Boost model: 0.9927678571428572
Average Recall for Gradient Boost: 0.8070535714285714


In [ ]:
#Based on the above recall scores, I would continue with the Ada Boost
#model.
#c. This model has almost a perfect recall (>80%), therefore not needing to 
    #tweak it. If we realized it was overfitting and that's why it's so high,
    #we coudl scale down on the trees or decrease the learning rate.